Comparing RTX5090-32GB results from different operating system (Windows11/WSL(Debian)/Linux(Fedora)). Using A100-SXM4-40GB results as reference.

You need to change output dir `output_folder`.

In [ ]:
import pandas as pd
import glob
import plotly.express as px
import plotly.graph_objects as go
import torchvision.models as models
output_folder = "fig/5090/"

You need to include the data you want in `CSV_DIC`.

In [ ]:
MODEL_DIC = {
    'mnasnet': models.mnasnet.__all__[1:],
    'resnet': models.resnet.__all__[1:],
    'densenet': models.densenet.__all__[1:],
    'squeezenet': models.squeezenet.__all__[1:],
    'vgg': models.vgg.__all__[1:],
    'mobilenet': models.mobilenet.__all__[1:],
    'shufflenetv2': models.shufflenetv2.__all__[1:]
}
model_list = MODEL_DIC.keys()

REV_MODEL_DIC = {}
for key, values in MODEL_DIC.items():
    for v in values:
        REV_MODEL_DIC[v] = key

BENCHMARK_TYPE = {
    'half_model_train_benchmark',
    'half_model_inference_benchmark',    
    'float_model_train_benchmark',
    'float_model_inference_benchmark',
    'double_model_train_benchmark',
    'double_model_inference_benchmark'
}

CSV_DIC = {
    'RTX5090/WIN11': glob.glob('results/5090/Win11/'+'*.csv'),
    'RTX5090/WSL': glob.glob('results/5090/WSL/'+'*.csv'),
    'RTX5090/Fedora': glob.glob('results/5090/Fedora/'+'*.csv'),
    'A100-SXM4-40GB': glob.glob('results/dgx-a100/'+'A100-SXM4-40GB_1*.csv')
}
gpu_list = CSV_DIC.keys()


In [ ]:
data_buffer = []
for btype in BENCHMARK_TYPE:
    for gpu in gpu_list:
        found_csv = [item for item in CSV_DIC[gpu] if btype in item]
        df = pd.read_csv(found_csv[0])
        df['run'] = df.index + 1
        df['gpu'] = gpu
        df['benchmark_type'] = btype
        df_melted = df.melt(id_vars=['run', 'gpu', 'benchmark_type'], var_name='model', value_name='latency')
        df_melted['model_type'] = df_melted['model'].map(REV_MODEL_DIC)
        data_buffer.append(df_melted)
all_data = pd.concat(data_buffer, ignore_index=True)
all_data = all_data.set_index(['benchmark_type', 'model_type'])
all_data = all_data.sort_index()


Preview a figure for testing.

In [ ]:
fig = px.box(
    all_data.loc[('double_model_train_benchmark', 'resnet')],
    y="latency",
    color="gpu",
    facet_col="model",  # facet_col / facet_row
    title="Latency by Model and GPU/" + "double_model_train_benchmark"
)
fig.update_traces(boxmean=True)
#fig.update_yaxes(dtick=50)

for annotation in fig.layout.annotations:
    annotation.text = annotation.text.replace("model=", "")
    # annotation.textangle = -15

fig.show()
#fig.write_image("output.png", width=3000, height=1000)
#fig.write_image("output.png", width=1500, scale=2)

Generate all figures.

In [ ]:
for idx in all_data.index.unique().tolist():
    fig = px.box(
        all_data.loc[idx],
        y="latency",
        color="gpu",
        facet_col="model",  # facet_col / facet_row
        title="Latency by Model and GPU/" + idx[0]
    )
    fig.update_traces(boxmean=True)
    for annotation in fig.layout.annotations:
        annotation.text = annotation.text.replace("model=", "")
    fig.write_image(output_folder+idx[0]+'_'+idx[1]+".png", width=1500, scale=2)